# Setup

In [1]:
import pandas as pd
import numpy as np
from langdetect import detect, DetectorFactory

In [2]:
DetectorFactory.seed = 0

In [3]:
na_vals = [R"\N"]
basics_r = pd.read_csv("basics.tsv", sep="\t", na_values=na_vals)
ratings_r = pd.read_csv("ratings.tsv", sep="\t",  na_values=na_vals)
crew_r = pd.read_csv("crew.tsv", sep="\t",  na_values=na_vals)
people_r = pd.read_csv("people.tsv", sep="\t", na_values=na_vals)

C:\Users\David\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
basics = basics_r[(basics_r["titleType"] == "movie") | (basics_r["titleType"] == "tvMovie")]
ratings = ratings_r
crew = crew_r
people = people_r

# displaying

In [5]:
# display("basics", basics)
# display("ratings", ratings)
# display("crew", crew)
# display("people", people)

# Merging + some edits

In [6]:
table_r = basics.merge(ratings, on="tconst", how="inner").merge(crew, on="tconst", how="inner")

In [7]:
table = (
    table_r.dropna(subset=["isAdult", "startYear", "runtimeMinutes", "genres", "averageRating", "numVotes"])
           .drop(["endYear", "writers"], axis=1) # enYear are all na for movies
           .rename({"startYear": "year"}, axis=1)
)
table

,tconst,titleType,primaryTitle,originalTitle,isAdult,year,runtimeMinutes,genres,averageRating,numVotes,directors
0,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894.0,45.0,Romance,5.3,208,nm0085156
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897.0,100.0,"Documentary,News,Sport",5.3,490,nm0714557
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,"Action,Adventure,Biography",6.0,863,nm0846879
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907.0,90.0,Drama,5.0,21,nm0141150
8,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,"Adventure,Fantasy",5.0,70,"nm0091767,nm0877783"
...,...,...,...,...,...,...,...,...,...,...,...
354282,tt9916192,tvMovie,Danielle Darrieux: Il est poli d'être gai!,Danielle Darrieux: Il est poli d'être gai!,0.0,2019.0,53,Biography,7.5,19,nm2144818
354283,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0.0,2020.0,84,Thriller,5.8,1461,nm1480867
354284,tt9916362,movie,Coven,Akelarre,0.0,2020.0,92,"Drama,History",6.4,5663,nm1893148
354287,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0.0,2019.0,123,Drama,8.6,7,nm4457074


In [8]:
table["isTvMovie"] = table["titleType"] == "tvMovie"
table = table.drop(columns=["titleType"])

# Director age

adds a column of the directors age at the time of movie release

In [9]:
table["directors"] = table["directors"].str.split(',')

table_exploded = table.explode('directors')

director_names = (
    people.loc[:, ["nconst", "birthYear"]]
          .rename({"nconst": "directors"}, axis=1)
          .dropna()
)
display(director_names)

tse_merged = pd.merge(table_exploded, director_names, on="directors", how="inner")

tse_merged["meanDirectorAgeAtRelease"] = tse_merged["year"] - tse_merged["birthYear"]
meanDirectorAgeAtRelease = tse_merged.groupby("tconst").agg({"meanDirectorAgeAtRelease": "mean"}).reset_index()

table = table.merge(meanDirectorAgeAtRelease, on="tconst", how="inner")
display(table)

,directors,birthYear
0,nm0000001,1899.0
1,nm0000002,1924.0
2,nm0000003,1934.0
3,nm0000004,1949.0
4,nm0000005,1918.0
...,...,...
13152772,nm9993436,1915.0
13152853,nm9993526,1972.0
13152861,nm9993535,2006.0
13152982,nm9993675,1925.0


,tconst,primaryTitle,originalTitle,isAdult,year,runtimeMinutes,genres,averageRating,numVotes,directors,isTvMovie,meanDirectorAgeAtRelease
0,tt0000009,Miss Jerry,Miss Jerry,0.0,1894.0,45.0,Romance,5.3,208,[nm0085156],False,35.0
1,tt0000147,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897.0,100.0,"Documentary,News,Sport",5.3,490,[nm0714557],False,34.0
2,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,"Action,Adventure,Biography",6.0,863,[nm0846879],False,38.0
3,tt0000591,The Prodigal Son,L'enfant prodigue,0.0,1907.0,90.0,Drama,5.0,21,[nm0141150],False,42.0
4,tt0000679,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0.0,1908.0,120.0,"Adventure,Fantasy",5.0,70,"[nm0091767, nm0877783]",False,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...
176738,tt9914254,A Cherry Tale,Kirsebæreventyret,0.0,2019.0,85,Documentary,7.1,8,[nm1375922],False,47.0
176739,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,0.0,2019.0,98,"Drama,Family",3.8,218,[nm1832009],False,63.0
176740,tt9915946,Nuestra película,Nuestra película,0.0,1993.0,96,Documentary,7.0,10,[nm0652213],False,44.0
176741,tt9916270,Il talento del calabrone,Il talento del calabrone,0.0,2020.0,84,Thriller,5.8,1461,[nm1480867],False,43.0


# Sampling


In [10]:
vote_info = table["numVotes"].describe()
vote_info

count    1.767430e+05
mean     5.622710e+03
std      4.626976e+04
min      5.000000e+00
25%      2.800000e+01
50%      1.020000e+02
75%      5.260000e+02
max      2.838758e+06
Name: numVotes, dtype: float64

## sample 4 quartiles based on numVotes

In [11]:
N_SAMPLES = 10000
STATE = 69

table_sorted = table.dropna().sort_values(by="numVotes")
table_sorted["quartile"] = pd.qcut(table_sorted["numVotes"], 4, labels=False)

samples = []
for quartile in range(4):
    samples.append(table_sorted[table_sorted["quartile"] == quartile].sample(n=N_SAMPLES, random_state=STATE))


table_sampled = pd.concat(samples)

## trans

In [12]:
def get_lang(text):
    try:
        return detect(text)
    except:
        print(text, "error!")
        return "en"

table_sampled["language"] = table_sampled["originalTitle"].apply(get_lang)
table_sampled

420 error!
2017 error!
33.333 error!
17 error!
30/30 error!
2020 error!
21:37 error!
475 error!
09.12.04. error!
1968 error!
1066 error!
10.32 error!
1973 error!
24 error!
3 error!
1993 error!
1918 error!
3 error!
1921 error!
11/8/16 error!
11.11 error!
76-89-03 error!
1666 error!
'38 error!
61* error!
1987 error!
678 error!
7500 error!
1408 error!
3 error!
14+ error!
$9.99 error!
51 error!
'71 error!
13/13/13 error!
1776 error!
1976 error!
1 error!
21 error!
10 error!
31 error!
65 error!


,tconst,primaryTitle,originalTitle,isAdult,year,runtimeMinutes,genres,averageRating,numVotes,directors,isTvMovie,meanDirectorAgeAtRelease,quartile,language
161679,tt4545976,Mirage of Love,Mirage d'amour avec fanfare,0.0,2015.0,99.0,"Drama,History,Romance",6.8,26,[nm0865651],False,60.0,0,fr
109189,tt0420894,Seref kavgasi,Seref kavgasi,0.0,1966.0,75.0,"Crime,Drama",5.8,13,[nm0037571],False,35.0,0,no
156350,tt3336542,Johnny Moccasin,Johnny Moccasin,0.0,1956.0,26.0,Western,5.5,15,[nm0070670],True,51.0,0,en
130891,tt1337609,Dubai Love,Dubai Love,0.0,2009.0,86.0,Drama,4.4,14,[nm1376857],False,33.0,0,id
107359,tt0400945,The Witches of Pendle,The Witches of Pendle,0.0,1976.0,76.0,Horror,7.3,21,[nm0115100],True,52.0,0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129788,tt13074150,The Execution,Kazn,0.0,2021.0,130.0,"Crime,Mystery,Thriller",6.7,1639,[nm8046930],False,34.0,3,sw
122905,tt1084950,Rachel Getting Married,Rachel Getting Married,0.0,2008.0,113.0,"Drama,Romance",6.7,50821,[nm0001129],False,64.0,3,de
56022,tt0104115,Doctor Mordrid,Doctor Mordrid,0.0,1992.0,74.0,"Action,Fantasy,Horror",5.4,2215,"[nm0051649, nm0023929]",False,54.5,3,es
113679,tt0475169,13,13 Tzameti,0.0,2005.0,93.0,"Crime,Drama,Thriller",7.3,17922,[nm1201875],False,26.0,3,hu


## genres

In [13]:
def get_all_categoricals(cat_col):
    ret_list = set()
    tmp = cat_col.apply(lambda x : x.split(","))
    for i in tmp:
        for j in i:
            ret_list.add(j)
    return sorted(list(ret_list))


def encode_categoricals(row, cats : list, column="genres"):
    genre_row = np.zeros(len(cats))
    for genre in row[column].split(","):
        if genre in cats:
            genre_row[cats.index(genre)] = 1
    for i, genre in enumerate(cats):
        row[genre] = genre_row[i]
    return row

In [14]:
all_genres = get_all_categoricals(table_sampled["genres"])
table_sampled = table_sampled.apply(encode_categoricals, cats=all_genres, axis=1)
table_sampled

,tconst,primaryTitle,originalTitle,isAdult,year,runtimeMinutes,genres,averageRating,numVotes,directors,...,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
161679,tt4545976,Mirage of Love,Mirage d'amour avec fanfare,0.0,2015.0,99.0,"Drama,History,Romance",6.8,26,[nm0865651],...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
109189,tt0420894,Seref kavgasi,Seref kavgasi,0.0,1966.0,75.0,"Crime,Drama",5.8,13,[nm0037571],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156350,tt3336542,Johnny Moccasin,Johnny Moccasin,0.0,1956.0,26.0,Western,5.5,15,[nm0070670],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
130891,tt1337609,Dubai Love,Dubai Love,0.0,2009.0,86.0,Drama,4.4,14,[nm1376857],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107359,tt0400945,The Witches of Pendle,The Witches of Pendle,0.0,1976.0,76.0,Horror,7.3,21,[nm0115100],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129788,tt13074150,The Execution,Kazn,0.0,2021.0,130.0,"Crime,Mystery,Thriller",6.7,1639,[nm8046930],...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
122905,tt1084950,Rachel Getting Married,Rachel Getting Married,0.0,2008.0,113.0,"Drama,Romance",6.7,50821,[nm0001129],...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
56022,tt0104115,Doctor Mordrid,Doctor Mordrid,0.0,1992.0,74.0,"Action,Fantasy,Horror",5.4,2215,"[nm0051649, nm0023929]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113679,tt0475169,13,13 Tzameti,0.0,2005.0,93.0,"Crime,Drama,Thriller",7.3,17922,[nm1201875],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## languages

In [15]:
all_langs = get_all_categoricals(table_sampled["language"])
table_sampled = table_sampled.apply(encode_categoricals, column="language", cats=all_langs, axis=1)
table_sampled

,tconst,primaryTitle,originalTitle,isAdult,year,runtimeMinutes,genres,averageRating,numVotes,directors,...,ro,sk,sl,so,sq,sv,sw,tl,tr,vi
161679,tt4545976,Mirage of Love,Mirage d'amour avec fanfare,0.0,2015.0,99.0,"Drama,History,Romance",6.8,26,[nm0865651],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109189,tt0420894,Seref kavgasi,Seref kavgasi,0.0,1966.0,75.0,"Crime,Drama",5.8,13,[nm0037571],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156350,tt3336542,Johnny Moccasin,Johnny Moccasin,0.0,1956.0,26.0,Western,5.5,15,[nm0070670],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130891,tt1337609,Dubai Love,Dubai Love,0.0,2009.0,86.0,Drama,4.4,14,[nm1376857],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107359,tt0400945,The Witches of Pendle,The Witches of Pendle,0.0,1976.0,76.0,Horror,7.3,21,[nm0115100],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129788,tt13074150,The Execution,Kazn,0.0,2021.0,130.0,"Crime,Mystery,Thriller",6.7,1639,[nm8046930],...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
122905,tt1084950,Rachel Getting Married,Rachel Getting Married,0.0,2008.0,113.0,"Drama,Romance",6.7,50821,[nm0001129],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56022,tt0104115,Doctor Mordrid,Doctor Mordrid,0.0,1992.0,74.0,"Action,Fantasy,Horror",5.4,2215,"[nm0051649, nm0023929]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113679,tt0475169,13,13 Tzameti,0.0,2005.0,93.0,"Crime,Drama,Thriller",7.3,17922,[nm1201875],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
table_sampled.columns

Index(['tconst', 'primaryTitle', 'originalTitle', 'isAdult', 'year',
       'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'directors',
       'isTvMovie', 'meanDirectorAgeAtRelease', 'quartile', 'language',
       'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War',
       'Western', 'af', 'ca', 'cs', 'cy', 'da', 'de', 'en', 'es', 'et', 'fi',
       'fr', 'hr', 'hu', 'id', 'it', 'lt', 'lv', 'nl', 'no', 'pl', 'pt', 'ro',
       'sk', 'sl', 'so', 'sq', 'sv', 'sw', 'tl', 'tr', 'vi'],
      dtype='object')

In [17]:
table_out = table_sampled.drop(columns=["originalTitle", "directors", "genres", "language", "quartile"])
display(table_out)
table_out.to_csv(f"output/data-{N_SAMPLES*4}.csv", index=False)

,tconst,primaryTitle,isAdult,year,runtimeMinutes,averageRating,numVotes,isTvMovie,meanDirectorAgeAtRelease,Action,...,ro,sk,sl,so,sq,sv,sw,tl,tr,vi
161679,tt4545976,Mirage of Love,0.0,2015.0,99.0,6.8,26,False,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109189,tt0420894,Seref kavgasi,0.0,1966.0,75.0,5.8,13,False,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156350,tt3336542,Johnny Moccasin,0.0,1956.0,26.0,5.5,15,True,51.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130891,tt1337609,Dubai Love,0.0,2009.0,86.0,4.4,14,False,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107359,tt0400945,The Witches of Pendle,0.0,1976.0,76.0,7.3,21,True,52.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129788,tt13074150,The Execution,0.0,2021.0,130.0,6.7,1639,False,34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
122905,tt1084950,Rachel Getting Married,0.0,2008.0,113.0,6.7,50821,False,64.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56022,tt0104115,Doctor Mordrid,0.0,1992.0,74.0,5.4,2215,False,54.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113679,tt0475169,13,0.0,2005.0,93.0,7.3,17922,False,26.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
